In [2]:
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import particles

from scipy.linalg import block_diag
from particles.collectors import Moments
from particles import state_space_models as ssm
from particles import distributions as dists

In [5]:
# to be removed
import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=119
)

<IPython.core.display.Javascript object>

# Two leg motion model
We consider a motion model, consisting of four rigid parts (left fibula, left femur, right fibula, right femur) and three hinges (left knee, hip, right knee). For simplicity, we assume the motion happens in the saggital plane, i.e. the motion is 2-dimensional:
<img src='images/TwoLegModel.png' width="500" height="500">

### States
We use the following states (blue), which describe the motion of this two leg model:

- position of the hip $ (x_H, y_H) $
- angle between vertical and left femur $ \varphi_0 $
- angle between left femur and left fibula $ \varphi_1 $
- angle between vertical and right femur $ \varphi_2 $
- angle between right femur and right fibula $ \varphi_3 $
- first and second derivatives of the above quantities: $ \dot x_H, \dot y_H, \ldots , \ddot \varphi_2, \ddot \varphi_3 $

In total, the state space is 36-dimensional $ x_t = (x_H, y_H, \varphi_0, \ldots \varphi_2, \ddot \varphi_3)^{T} \in \mathbb{R}^{36} $.

### Observations
As observations (orange), we use four inertial measurement units (IMU) and two pressure sensors at the feet. Each IMU measures linear accelerations $ \ddot x $- and $  \ddot y $ and angular velocity $ \omega $ in the $ x $-$ y $-plane:
$$
y^{\text{IMU}_{i}} = (\ddot x^{(i)}, \ddot y^{(i)}, \dot \omega^{(i)}), \quad i=0, 1, 2, 3
$$
The two pressure sensors only detect if there is ground contact or not. We can compute the expected velocity and acceleration from the states. If a sensor signals ground contact, we expect our prediction for velocity and acceleration to be close to 0. Another implication is that there will be times, when we have no signal from a pressure sensor, i.e. missing observations.
For each feet, we have four observation variables:
$$
y^{\text{PS}_{i}} = (\dot x^{(i)}, \dot y^{(i)}, \ddot x^{(i)}, \ddot y^{(i)}), \quad i=4, 5
$$
In total, the observation space is $ 4 \cdot 3 + 2 \cdot 8 = 20 $-dimensional $ y_{t} = (\ddot x^{(0)}, \ddot y^{(0)}, \dot \omega^{(0)}, \ldots , \dot x^{(5)}, \dot y^{(5)}, \ddot x^{(5)}, \ddot y^{(5)})^T \in \mathbb{R}^{20} $

### Data
Lets load the data. In this case, the data was generated using a simulator. This has the advantage, that observations as well as states are available. We use data without any missing data points; under "data/missingObservations/" there is more realistic data, where some data from the pressure sensors at the feet is missing.

In [24]:
true_states = pd.read_csv("data/normal/states.csv")
observations = pd.read_csv("data/normal/observations.csv")

In [25]:
true_states.head()

,$x_H$,$y_H$,$\varphi_0$,$\varphi_1$,$\varphi_2$,$\varphi_3$,$\dot x_H$,$\dot y_H$,$\dot \varphi_0$,$\dot \varphi_1$,$\dot \varphi_2$,$\dot \varphi_3$,$\ddot x_H$,$\ddot y_H$,$\ddot \varphi_0$,$\ddot \varphi_1$,$\ddot \varphi_2$,$\ddot \varphi_3 $
0,0.005679,1.057527,-0.128463,-0.247931,0.366397,-0.189801,0.567903,0.096320,2.536291,-3.798610,-0.078163,-0.818193,-4.070523e-11,0.005052,-1.776230,3.315853,-0.295288,0.535814
1,0.011358,1.058492,-0.103236,-0.285663,0.365574,-0.197908,0.567903,0.096974,2.504285,-3.738704,-0.089268,-0.798176,-1.251775e-10,0.128222,-4.654972,8.721949,-1.955715,3.521541
2,0.017037,1.059471,-0.078476,-0.322520,0.364548,-0.205651,0.567903,0.099096,2.442664,-3.623065,-0.119473,-0.743827,-2.756526e-10,0.298159,-7.674005,14.416044,-4.105452,7.384565
3,0.022716,1.060479,-0.054470,-0.357959,0.363123,-0.212674,0.567903,0.102600,2.354972,-3.458162,-0.167874,-0.656751,-5.076412e-10,0.397818,-9.805368,18.454107,-5.525057,9.941562
4,0.028395,1.061526,-0.031434,-0.391573,0.361152,-0.218716,0.567903,0.106852,2.250174,-3.260731,-0.227607,-0.549239,-8.333688e-10,0.448776,-11.084394,20.902107,-6.375846,11.479190


In [27]:
observations.head()

,$\ddot x^0$,$\ddot y^0$,$\omega^0$,$\ddot x^1$,$\ddot y^1$,$\omega^1$,$\ddot x^2$,$\ddot y^2$,$\omega^2$,$\ddot x^3$,$\ddot y^3$,$\omega^3$,$\dot x^4$,$\dot y^4$,$\ddot x^4$,$\ddot y^4$,$\dot x^5$,$\dot y^5$,$\ddot x^5$,$\ddot y^5$
0,-1.821814,11.900176,2.550071,-4.852858,12.373303,-1.280894,3.385255,8.980552,-0.080017,1.492541,9.627988,-0.875733,1.121228,0.212252,0.741781,3.858301,0.001966,-0.012164,-0.081558,0.454939
1,-2.361384,11.853904,2.514129,-6.011847,11.825068,-1.240908,3.043565,8.880006,-0.092579,1.060646,9.790665,-0.871481,1.128034,0.248770,0.612685,3.407860,0.001219,-0.007841,-0.067200,0.405066
2,-2.962156,11.609948,2.441278,-6.424099,11.467285,-1.204667,2.108684,9.413049,-0.110925,0.337125,9.618804,-0.888815,1.133419,0.279801,0.459334,2.763782,0.000638,-0.004161,-0.048449,0.325825
3,-4.042005,11.968531,2.381178,-6.927154,10.534151,-1.080061,2.099710,9.573533,-0.156228,0.342423,9.868236,-0.789812,1.137230,0.303813,0.302377,2.022525,0.000253,-0.001372,-0.028578,0.230321
4,-3.883105,11.748364,2.242771,-7.394055,9.943406,-1.029109,1.617350,9.982116,-0.225111,-0.265758,9.721602,-0.793610,1.139544,0.320394,0.163058,1.293955,0.000054,0.000473,-0.011828,0.139640


### Mechanical model
We need to compute the state transition and to compute the observations from the states. This can be one using basic mechanics.

For the state transition, we assume a uniformly accelerated motion for each time step. For the $ x $-coordinate of the hip this means
$$
x_H(t) = x_H(t-1) + \Delta t \cdot \dot x_H(t-1) + \frac{1}{2} \Delta t^{2} \cdot \ddot x_H(t-1)^{2}
$$
This might be a reasonable assumption, if the time delta $ \Delta t $ is small enough. The transition of the state vector $ x_{t} $ can then be described using a linear transformation
$$
x_{t} = A \cdot x_{t-1}
$$

The state to observation map $ h \colon x_{t} \mapsto y_{t} $ is highly non-linear. In the cell below, the necessary functions for the state to observation computations are defined.


For more details on these maps, see [Reference MT].

In [28]:
def state_to_obs(x, dim_observations, g, legs, imus):
    nb_steps, _ = x.shape
    y = np.empty((nb_steps, dim_observations))
    # left femur
    y[:, 0] = imus[0] * x[:, 14] + g * np.sin(x[:, 2]) + np.sin(x[:, 2]) * x[:, 13] + np.cos(x[:, 2]) * x[:, 12]
    y[:, 1] = imus[0] * x[:, 8] ** 2 + g * np.cos(x[:, 2]) - np.sin(x[:, 2]) * x[:, 12] + np.cos(x[:, 2]) * x[:, 13]
    y[:, 2] = x[:, 8]

    # left fibula
    y[:, 3] = (
        imus[1] * x[:, 14]
        + imus[1] * x[:, 15]
        + g * np.sin(x[:, 2] + x[:, 3])
        + legs[0] * np.sin(x[:, 3]) * x[:, 8] ** 2
        + legs[0] * np.cos(x[:, 3]) * x[:, 14]
        + np.sin(x[:, 2] + x[:, 3]) * x[:, 13]
        + np.cos(x[:, 2] + x[:, 3]) * x[:, 12]
    )
    y[:, 4] = (
        imus[1] * x[:, 8] ** 2
        + 2 * imus[1] * x[:, 8] * x[:, 9]
        + imus[1] * x[:, 9] ** 2
        + g * np.cos(x[:, 2] + x[:, 3])
        - legs[0] * np.sin(x[:, 3]) * x[:, 14]
        + legs[0] * np.cos(x[:, 3]) * x[:, 8] ** 2
        - np.sin(x[:, 2] + x[:, 3]) * x[:, 12]
        + np.cos(x[:, 2] + x[:, 3]) * x[:, 13]
    )
    y[:, 5] = x[:, 8] + x[:, 9]

    # right femur
    y[:, 6] = imus[2] * x[:, 16] + g * np.sin(x[:, 4]) + np.sin(x[:, 4]) * x[:, 13] + np.cos(x[:, 4]) * x[:, 12]
    y[:, 7] = imus[2] * x[:, 10] ** 2 + g * np.cos(x[:, 4]) - np.sin(x[:, 4]) * x[:, 12] + np.cos(x[:, 4]) * x[:, 13]
    y[:, 8] = x[:, 10]

    # right fibula
    y[:, 9] = (
        imus[3] * x[:, 16]
        + imus[3] * x[:, 17]
        + g * np.sin(x[:, 4] + x[:, 5])
        + legs[2] * np.sin(x[:, 5]) * x[:, 10] ** 2
        + legs[2] * np.cos(x[:, 5]) * x[:, 16]
        + np.sin(x[:, 4] + x[:, 5]) * x[:, 13]
        + np.cos(x[:, 4] + x[:, 5]) * x[:, 12]
    )
    y[:, 10] = (
        imus[3] * x[:, 10] ** 2
        + 2 * imus[3] * x[:, 10] * x[:, 11]
        + imus[3] * x[:, 11] ** 2
        + g * np.cos(x[:, 4] + x[:, 5])
        - legs[2] * np.sin(x[:, 5]) * x[:, 16]
        + legs[2] * np.cos(x[:, 5]) * x[:, 10] ** 2
        - np.sin(x[:, 4] + x[:, 5]) * x[:, 12]
        + np.cos(x[:, 4] + x[:, 5]) * x[:, 13]
    )
    y[:, 11] = x[:, 10] + x[:, 11]

    # left heel
    y[:, 12] = (
        legs[0] * np.cos(x[:, 2]) * x[:, 8] + legs[1] * (x[:, 8] + x[:, 9]) * np.cos(x[:, 2] + x[:, 3]) + x[:, 6]
    )
    y[:, 13] = (
        legs[0] * np.sin(x[:, 2]) * x[:, 8] + legs[1] * (x[:, 8] + x[:, 9]) * np.sin(x[:, 2] + x[:, 3]) + x[:, 7]
    )
    y[:, 14] = (
        -legs[0] * np.sin(x[:, 2]) * x[:, 8] ** 2
        + legs[0] * np.cos(x[:, 2]) * x[:, 14]
        - legs[1] * (x[:, 8] + x[:, 9]) ** 2 * np.sin(x[:, 2] + x[:, 3])
        + legs[1] * (x[:, 14] + x[:, 15]) * np.cos(x[:, 2] + x[:, 3])
        + x[:, 12]
    )
    y[:, 15] = (
        legs[0] * np.sin(x[:, 2]) * x[:, 14]
        + legs[0] * np.cos(x[:, 2]) * x[:, 8] ** 2
        + legs[1] * (x[:, 8] + x[:, 9]) ** 2 * np.cos(x[:, 2] + x[:, 3])
        + legs[1] * (x[:, 14] + x[:, 15]) * np.sin(x[:, 2] + x[:, 3])
        + x[:, 13]
    )

    # right heel
    y[:, 16] = (
        legs[2] * np.cos(x[:, 4]) * x[:, 10] + legs[3] * (x[:, 10] + x[:, 11]) * np.cos(x[:, 4] + x[:, 5]) + x[:, 6]
    )
    y[:, 17] = (
        legs[2] * np.sin(x[:, 4]) * x[:, 10] + legs[3] * (x[:, 10] + x[:, 11]) * np.sin(x[:, 4] + x[:, 5]) + x[:, 7]
    )
    y[:, 18] = (
        -legs[2] * np.sin(x[:, 4]) * x[:, 10] ** 2
        + legs[2] * np.cos(x[:, 4]) * x[:, 16]
        - legs[3] * (x[:, 10] + x[:, 11]) ** 2 * np.sin(x[:, 4] + x[:, 5])
        + legs[3] * (x[:, 16] + x[:, 17]) * np.cos(x[:, 4] + x[:, 5])
        + x[:, 12]
    )
    y[:, 19] = (
        legs[2] * np.sin(x[:, 4]) * x[:, 16]
        + legs[2] * np.cos(x[:, 4]) * x[:, 10] ** 2
        + legs[3] * (x[:, 10] + x[:, 11]) ** 2 * np.cos(x[:, 4] + x[:, 5])
        + legs[3] * (x[:, 16] + x[:, 17]) * np.sin(x[:, 4] + x[:, 5])
        + x[:, 13]
    )

    return y

To define the state space model, we need to define the distributions for the state transition and for the transition from state to observations. We assume the following

$$
\text{Initialization: } X_{0} \sim \mathcal{N}(b_{0}, Q_{0}) \\
\text{State transition: } X_{t} | X_{t-1}=x_{t-1} \sim \mathcal{N}(A \cdot x_{t-1}, Q) \\
\text{State to observation: } Y_{t} | X_{t}=x_{t} \sim \mathcal{N}(h(x_{t}), V)
$$

where, $ \mathcal{N}(\mathbf{\mu}, \Sigma) $ denotes the multivariate normal distribution with mean $ \mu $ and covariance matrix $ \Sigma $.

We choose the covariance matrix for the initial distribution $ Q_{0} $ to be a diagonal matrix; as will be seen, the choice of the mean $ b_{0} $ is not too important.

The covariance matrix $ Q $ for the state transition is a band diagonal matrix. It arises from the fact that the state variables $ \ddot x_H, \ddot y_H, \ddot \varphi_{0}, \ddot \varphi_{1}, \ddot \varphi_{2}, \ddot \varphi_{3} $ are modeled as Browninan motion. Again, for more details see [Reference MT].

The covariance matrix $ V $ is diagonal with reasonable assumptions for the uncertainty of observations.

## Model parameters
We need to define some parameters for the model, in particular the lenght of the limbs and the position of the IMUs.
<img src='images/ModelParameter.png' width="500" height="500">
For simplicity, we choose the same parameters as in the data generator:

$$
l_0 = 0.5, l_1 = 0.6, l_0 = 0.5, l_1 = 0.6 \\
s_0 = 0.34, s_1 = 0.29, s_2 = 0.315, s_3 = 0.33
$$

With this parameters, we define the state space model class for the two leg motion model:

In [29]:
class TwoLegModel(ssm.StateSpaceModel):
    def __init__(
        self,
        dt=0.01,
        dim_states=18,
        dim_observations=20,
        femur_left=0.5,
        fibula_left=0.6,
        femur_right=0.5,
        fibula_right=0.6,
        pos_imu0=0.34,
        pos_imu1=0.29,
        pos_imu2=0.315,
        pos_imu3=0.33,
        b0=np.array([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
        factor_Q0=0.1,
        sigma_imu_acc=0.1,
        sigma_imu_gyro=0.1,
        sigma_press_velo=0.1,
        sigma_press_acc=1.0,
    ):
        ssm.StateSpaceModel().__init__()
        self.dt = dt
        self.dim_states = dim_states
        self.dim_observations = dim_observations
        self.A = np.zeros((dim_states, dim_states))
        self.set_process_transition_matrix()
        self.g = 9.81
        self.len_legs = np.array([femur_left, fibula_left, femur_right, fibula_right])
        self.pos_imus = np.array([pos_imu0, pos_imu1, pos_imu2, pos_imu3])
        self.b0 = b0
        self.factor_Q0 = factor_Q0
        self.Q0 = self.factor_Q0 * np.eye(self.dim_states)
        self.sigma_imu_acc = sigma_imu_acc
        self.sigma_imu_gyro = sigma_imu_gyro
        self.sigma_press_velo = sigma_press_velo
        self.sigma_press_acc = sigma_press_acc
        self.Q = np.zeros((self.dim_states, self.dim_states))
        self.set_process_covariance()
        self.V = np.zeros((self.dim_observations, self.dim_observations))
        self.set_observation_covariance()

    def set_process_transition_matrix(self):
        self.A = np.eye(self.dim_states)
        for row in range(0, self.dim_states):
            for col in range(0, self.dim_states):
                if row + 6 == col:
                    self.A[row, col] = self.dt
                if row + 12 == col:
                    self.A[row, col] = self.dt**2 / 2.0

    def set_process_covariance(self):
        block_size = self.dim_states // 3
        for row in range(0, self.dim_states):
            for col in range(0, self.dim_states):
                if row < block_size:
                    if row == col:
                        self.Q[row, col] = self.dt**5 / 20.0
                    elif row + 6 == col:
                        self.Q[row, col] = self.dt**4 / 8.0
                        self.Q[col, row] = self.dt**4 / 8.0
                    elif row + 12 == col:
                        self.Q[row, col] = self.dt**3 / 6.0
                        self.Q[col, row] = self.dt**3 / 6.0
                elif block_size <= row < 2 * block_size:
                    if row == col:
                        self.Q[row, col] = self.dt**3 / 3.0
                    elif row + 6 == col:
                        self.Q[row, col] = self.dt**2 / 2.0
                        self.Q[col, row] = self.dt**2 / 2.0
                elif 2 * block_size <= row:
                    if row == col:
                        self.Q[row, col] = self.dt

    def set_observation_covariance(self):
        self.V = np.diag(
            [
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_press_velo,
                self.sigma_press_velo,
                self.sigma_press_acc,
                self.sigma_press_acc,
                self.sigma_press_velo,
                self.sigma_press_velo,
                self.sigma_press_acc,
                self.sigma_press_acc,
            ]
        )

    def state_transition(self, xp):
        return np.matmul(self.A, xp.T).T

    def state_to_observation(self, x):
        return state_to_obs(x, self.dim_observations, self.g, self.len_legs, self.pos_imus)

    def PX0(self):
        return dists.MvNormal(loc=self.b0, cov=self.Q0)

    def PX(self, t, xp):
        return dists.MvNormal(loc=self.state_transition(xp), cov=self.Q)

    def PY(self, t, xp, x):
        return dists.MvNormal(loc=self.state_to_observation(x), cov=self.V)

### Bootstrap particle filter
As first step, we implement a bootstrap particle filter. We run the particle filter with 500 particles and a essential sample size of 0.5. For the model, we use the default parameters. We keep track of the time, the resample steps and the log likelihood:

In [30]:
model = TwoLegModel()

nb_particles = 500
nb_timesteps = observations.shape[0]
ESSrmin = 0.5

fk_model = ssm.Bootstrap(ssm=model, data=observations.to_numpy())

pf = particles.SMC(
    fk=fk_model,
    N=nb_particles,
    ESSrmin=ESSrmin,
    store_history=True,
    collect=[Moments()],
)
start_user, start_process = time.time(), time.process_time()
pf.run()
end_user, end_process = time.time(), time.process_time()
s_user = end_user - start_user
s_process = end_process - start_process
print(
    f"Time user {s_user // 60:.0f}min {s_user % 60:.0f}s; time processor {s_process // 60:.0f}min {s_process % 60:.0f}s"
)
print(f"Resampled {np.sum(pf.summaries.rs_flags)} of totally {nb_timesteps} steps.")
loglikelihood = pf.summaries.logLts[-1]
print(f"Log likelihood = {loglikelihood:.3E}")

Time user 0min 3s; time processor 0min 20s
Resampled 997 of totally 1000 steps.
Log likelihood = -1.895E+08


The number of resampling steps and the log likelihood already indiacte a poor performance of the bootstrap filter. Indeed, if we compare the particle trajectories and the true states, we get the following picture.

In [31]:
def plot_results(pf, obs_map, x, y, dt, export_name, show_fig, plt_smthng=False):
    plotter = Plotter(
        true_states=np.array(x), true_obs=np.array(y), delta_t=dt, show_fig=show_fig, export_name=export_name
    )

    plotter.plot_observations(np.array(pf.hist.X), observation_map=obs_map)
    plotter.plot_particles_trajectories(np.array(pf.hist.X))
    particles_mean = np.array([m["mean"] for m in pf.summaries.moments])
    particles_var = np.array([m["var"] for m in pf.summaries.moments])
    plotter.plot_particle_moments(particles_mean=particles_mean, particles_var=particles_var)
    plotter.plot_ESS(pf.summaries.ESSs)
    plotter.plot_logLts_one_run(pf.summaries.logLts)
    if plt_smthng:
        smooth_trajectories = pf.hist.backward_sampling(10, linear_cost=False, return_ar=False)
        data_writer = DataReaderWriter()
        data_writer.export_trajectory(np.array(smooth_trajectories), dt, export_name)
        plotter.plot_smoothed_trajectories(samples=np.array(smooth_trajectories))
    return None

In [41]:
from Plotter import Plotter

dt = 0.01
show_fig = True
plt_smoothed = True  # set to False if no smoothing wanted
export_name = "test"

plot_results(pf, model.state_to_observation, true_states, observations, dt, export_name, show_fig, plt_smoothed)

ValueError: not enough values to unpack (expected 3, got 2)